## Imports

In [4]:
import googlemaps
import pandas as pd
import numpy as np
import os
import folium
import glob

## Read-in

In [5]:
df_listed = pd.read_csv('Dev sites for sale _ recently sold - MAP - Listings.csv')
df_sold = pd.read_csv('Dev sites for sale _ recently sold - MAP - Recent sales.csv')

## Data prep

In [13]:
df_concat = pd.concat([df_listed,df_sold])
df_concat.reset_index(drop=True,inplace=True)

In [14]:
df_concat

,full_address,status,Owner/seller,Description,Neighborhood,Asking price,Previous sale price,story_link,extra info,other_addresses,Sale price,Sale date
0,29 NW 28th Street in Miami,listed,"29 NW 28 STREET LLC, led by Lance Burstyn and ...",The 0.2-acre site is zoned for 24 residential ...,Wynwood,$5M,$1.1M in 2018,NaN,NaN,"60 NE 23rd St, Miami, FL 33137 68 NE 23rd St, ...",NaN,NaN
1,2214 North Miami Avenue in Miami,listed,2214 N MIAMI AVE DEVELOPMENT LLC/ 7G REALTY MA...,"The 0.4-acre lot is zoned T6-8-O, which allows...",Wynwood,$18.8M,$10.8M in 2022,https://therealdeal.com/miami/2022/03/31/east-...,NaN,NaN,NaN,NaN
2,201 SW 8th Street in Miami,listed,Two entities managed by Simon and Antoine Karam,The properties have recently realized Rapid Tr...,Brickell,NaN,$19.5M,NaN,NaN,720 SW 2nd Avenue,NaN,NaN
3,1501 Northwest 37th Street in Miami,listed,"Josa Property 15, led by Jorge Salazar","The 1.5-acre assemblage is zoned T6-8-O, which...",Allapattah,$11.5M,NaN,NaN,https://www.crexi.com/properties/1281366/flori...,NaN,NaN,NaN
4,"3302 & 3312 NW 2nd Avenue, Miami",listed,NaN,"Colliers, as the exclusive listing broker, is ...",Wynwood,NaN,NaN,NaN,https://www.crexi.com/properties/1403922/flori...,NaN,NaN,NaN
5,"2200 NW 2ND AVE.\nMIAMI, FLORIDA\n33127",listed,Brooklyn-based ABINGDON SQUARE PARTNERS,The offering presents one of the last\nopportu...,Wynwood,$31M,$5.1M,NaN,https://docs.google.com/spreadsheets/d/1OWoUw_...,2230 NW 2ND AVE. 224 NW 22ND TER.,NaN,NaN
6,SW 216 Street and SW 119 Avenue,listed,NaN,"Colliers is pleased to present Goulds 255, a r...",Unincorporated Miami-Dade,$4.5M,NaN,NaN,https://www.crexi.com/properties/1416324/flori...,NaN,NaN,NaN
7,"459 NE 25th Street, Edgewater, Miami",listed,"\nHABITUS EDGEWATER HF1 B, affiliate of Miami-...",The Porosoff Group of Compass Commercial is pr...,Edgewater,$5.1M,$1.9M,NaN,NaN,NaN,NaN,NaN
8,"523 & 545 NE 125 St, North Miami",listed,Homestead-based STEMBRIDGE REAL ESTATE CO,Gridline Properties is pleased to bring to mar...,North Miami,$6M,NaN,NaN,NaN,544 NE 126 St,NaN,NaN
9,"3728 NW 19th Avenue, Allapattah, Florida",listed,TROPIC PLACE VENTURES managed by Gerard Fernan...,Truly shovel ready\nAll impact fees paid ($115...,Allapattah,$1.3M,$440K,NaN,NaN,NaN,NaN,NaN


## Geocode

In [15]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [16]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df_concat['geocoded'] = df_concat['full_address'].apply(geocode)

In [18]:
df = df_concat

In [19]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [9]:
# df.at[index#,'col_name']

## Format Data

In [20]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

## Prepare Data

In [22]:
df['status'].unique()

array(['listed', 'sold'], dtype=object)

In [23]:
df['color'] = ''

df.loc[df['status'] == 'listed', 'color'] = 'red'
df.loc[df['status'] == 'sold', 'color'] = 'blue'

## HTML Popup Formatter

In [25]:
df.columns

Index(['full_address', 'status', 'Owner/seller', 'Description', 'Neighborhood',
       'Asking price', 'Previous sale price', 'story_link', 'extra info',
       'other_addresses', 'Sale price', 'Sale date', 'geocoded', 'lat', 'lon',
       'color'],
      dtype='object')

In [26]:
df.head(1)

,full_address,status,Owner/seller,Description,Neighborhood,Asking price,Previous sale price,story_link,extra info,other_addresses,Sale price,Sale date,geocoded,lat,lon,color
0,29 NW 28th Street in Miami,listed,"29 NW 28 STREET LLC, led by Lance Burstyn and ...",The 0.2-acre site is zoned for 24 residential ...,Wynwood,$5M,$1.1M in 2018,NaN,NaN,"60 NE 23rd St, Miami, FL 33137 68 NE 23rd St, ...",NaN,NaN,"(25.8032799, -80.1956357)",25.80328,-80.195636,red


In [35]:
def popup_html(row):
    html_parts = []
    html_parts.append('<!DOCTYPE html><html><head><style>.popup-content {max-width: 256px; word-wrap: break-word;}</style></head><body><div class="popup-content">')

    # List of tuples containing the label and value
    info = [
        ('Address', row['full_address']),
        ('Status', row['status']),
        ('Neighborhood', row['Neighborhood']),
        ('Asking Price', row['Asking price']),
        ('Previous Sale Price', row['Previous sale price']),
        ('Sale Price', row['Sale price']),
        ('Sale Date', row['Sale date'])
    ]

    for label, value in info:
        if pd.notna(value):  # Check if the value is not NaN
            html_parts.append(f"<strong>{label}: </strong>{value}<br>")

    html_parts.append('</div></body></html>')
    
    return ''.join(html_parts)

df['legend_text'] = df['status'] + ' (' + df['color'] + ')'

In [41]:
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format('South Florida Development Sites Listed and Sold')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9.5, tiles=None)

# Initialize a dictionary to hold the FeatureGroups for each status
status_groups = {}

for index, row in df.iterrows():
    status = row['legend_text']
    icon_color = row['color']  # Assuming you still want to use color for marker icons
    
    # Check if the FeatureGroup for this status already exists; if not, create it
    if status not in status_groups:
        status_groups[status] = folium.FeatureGroup(name=status).add_to(m)
    
    # Create a Marker for each row
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html(row), max_width=400),
        icon=folium.Icon(color=icon_color)  # Use the 'color' value for the icon
    )
    
    # Add the marker to the corresponding status group
    marker.add_to(status_groups[status])

# Add a LayerControl to allow users to toggle status groups
folium.LayerControl(collapsed=False).add_to(m)
folium.TileLayer('OpenStreetMap',control=False).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

# Display map
m

In [42]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SouthFloridaDevSitesListed_Sold
